In [1]:
# initial set-up
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset
import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from utilities3 import *

from Adam import Adam
from sewar.full_ref import rmse, uqi
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import timeit

torch.manual_seed(0)
np.random.seed(0)

c:\Users\azhir\anaconda3\envs\gundam\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# MODEL METRICS
###################################
# MODEL METRICS
###################################
def basic_metrics(recon_data,true_data, no_recon):
    rms_values, psnr_values, ssim_values, uqi_values = [], [], [], []

    for test_num in range(no_recon):
        predicted_np = np.reshape(recon_data[test_num,:],(16,16))
        truth_np = np.reshape(true_data[test_num,:],(16,16))

        rms_values.append(rmse(predicted_np, truth_np))
        psnr_values.append(psnr(truth_np, predicted_np, data_range=predicted_np.max() - predicted_np.min()))
        ssim_values.append(ssim(truth_np, predicted_np, data_range=predicted_np.max() - predicted_np.min()))
        uqi_values.append(uqi(predicted_np, truth_np))

    model_rms = sum(rms_values)/ len(rms_values)
    std_rms = np.std(np.array(rms_values))

    model_psnr = sum(psnr_values)/ len(psnr_values)
    std_psnr = np.std(np.array(psnr_values))

    model_ssim = sum(ssim_values)/ len(ssim_values)
    std_ssim = np.std(np.array(ssim_values))

    model_uqi = sum(uqi_values)/ len(uqi_values)
    std_uqi = np.std(np.array(uqi_values))


    print("RMSE: ", model_rms, std_rms, sep="---")
    print("PSNR: ", model_psnr, std_psnr, sep="---")
    print("SSIM: ", model_ssim, std_ssim, sep="---")

    output = {
    "rms": rms_values,
    "psnr": psnr_values,
    "ssim": ssim_values,
    # average values
    'model_rms': [round(model_rms,5),round(std_rms,5)],
    'model_psnr': [round(model_psnr,5),round(std_psnr,5)],
    'model_ssim': [round(model_ssim,5),round(std_ssim,5)]
    }
    return output 

In [9]:
measurement_list = ['recon000', 'recon005', 'recon010', 'recon015', 'recon020', 'recon025', 'recon030']
noise_levels = [0, 0.005, 0.010, 0.015, 0.020, 0.025, 0.030]

In [65]:
gn_raw_data = scipy.io.loadmat('./datasets/recon_info_v2')
truth_gn = np.asarray(gn_raw_data['solution_data'])

gn_stats = scipy.io.loadmat('./datasets/recon_means_finalv2')

In [66]:
mean_itr = np.asarray(gn_stats['mean_itr'][0])
std_itr = np.asarray(gn_stats['std_itr'][0])

mean_time = np.asarray(gn_stats['mean_time'][0])
std_time = np.asarray(gn_stats['std_time'][0])

In [70]:
std_time, mean_time

(array([3.86754111, 8.20684976, 6.12760351, 5.54010804, 6.17872694,
        5.10849386, 5.39823338]),
 array([25.89124692, 27.74351994, 23.66078866, 19.200918  , 16.01148558,
        12.16665366, 10.92320083]))

In [69]:
std_itr, mean_itr

(array([3.31319675, 6.59031276, 5.01604496, 4.83475423, 5.29913277,
        4.49259773, 4.75568772]),
 array([22.55, 23.39, 20.03, 16.67, 13.8 , 10.72,  9.64]))

In [31]:
noise = 6

truth_gn = np.asarray(gn_raw_data['solution_data'])
recon_gn = np.asarray(gn_raw_data[measurement_list[noise]])

gn_metrics = basic_metrics( recon_gn, truth_gn, truth_gn.shape[0] )

RMSE: ---0.006460539945241115---0.0010965694762980463
PSNR: ---20.197436717593877---1.595893637789947
SSIM: ---0.6282667556705349---0.13365973825350877


In [71]:
output_file_name = 'gn_recon_results2.txt'
with open(output_file_name, 'w') as output:
    output.write('textbf{ Model 4} & textbf{ Noise Level } & textbf{Itr No} & textbf{Recon Time /sec}  & textbf{RMSE} & textbf{PSNR} & textbf{SSIM} \n')

In [72]:
for i in range(7):

    print('noise levels:', noise_levels[i])

    truth_gn = np.asarray(gn_raw_data['solution_data'])
    recon_gn = np.asarray(gn_raw_data[measurement_list[i]])

    gn_metrics = basic_metrics(truth_gn, recon_gn, truth_gn.shape[0] )

    ###########################################################
    # Open a file with access mode 'a'
    file_object = open(output_file_name, 'a')
    # Append 'hello' at the end of file
    info = 'GN | ' + ' & ' + str(noise_levels[i]) + ' & ' + str(round(mean_itr[i],2)) + ' \ textpm ' + str(round(std_itr[i],2)) + ' & '+ str(round(mean_time[i],2)) + ' \ textpm ' + str(round(std_time[i],2)) + ' & '
    info_rms = str(gn_metrics.get('model_rms')[0]) + ' \ textpm ' + str(gn_metrics.get('model_rms')[1]) + '  &  '
    info_psnr = str(gn_metrics.get('model_psnr')[0]) + ' \ textpm ' + str(gn_metrics.get('model_psnr')[1]) + '  &  '
    info_ssim = str(gn_metrics.get('model_ssim')[0]) + ' \ textpm ' + str(gn_metrics.get('model_ssim')[1]) + '  \ \  ' + '\n'

    file_object.write(info + info_rms + info_psnr + info_ssim)
    # Close the file
    file_object.close()

noise levels: 0
RMSE: ---0.002126769821239712---0.0010751243920038114
PSNR: ---27.55968813673472---3.270859093169652
SSIM: ---0.9120946800401533---0.029141184757937876
noise levels: 0.005
RMSE: ---0.002532853875159845---0.0009469211681225116
PSNR: ---25.436127994083662---2.026794346920264
SSIM: ---0.8707387035745963---0.05766196204568056
noise levels: 0.01
RMSE: ---0.0034055384415148625---0.0008300492885507383
PSNR: ---22.496028072720264---1.951590032046097
SSIM: ---0.7956642796877724---0.099433352722498
noise levels: 0.015
RMSE: ---0.004270500600128336---0.0007294567573465639
PSNR: ---20.400193580904382---2.026507107843556
SSIM: ---0.7312914740319164---0.1230194557603316
noise levels: 0.02
RMSE: ---0.005171009979237973---0.0009707666156347102
PSNR: ---18.759777824141555---2.537701000341775
SSIM: ---0.6842881071094462---0.13390849462896065
noise levels: 0.025
RMSE: ---0.00572828675206471---0.0009249365819147889
PSNR: ---17.83087201691088---2.282764285080629
SSIM: ---0.6479082310080864-